In [1]:
from bloqade.gemini import logical as gemini_logical
from bloqade import qubit, squin
from bloqade.lanes.logical_mvp import (
    compile_to_physical_stim_program,
    kernel,
    set_detector,
    set_observable,
    TesseractDecoder,
    GeminiLogical,
)

In [2]:
@kernel
def main():
    # see arXiv: 2412.15165v1, Figure 3a
    reg = qubit.qalloc(5)
    squin.broadcast.t(reg)

    squin.broadcast.sqrt_x([reg[0], reg[1], reg[4]])
    squin.broadcast.cz([reg[0], reg[2]], [reg[1], reg[3]])
    squin.broadcast.sqrt_y([reg[0], reg[3]])
    squin.broadcast.cz([reg[0], reg[3]], [reg[2], reg[4]])
    squin.sqrt_x_adj(reg[0])
    squin.broadcast.cz([reg[0], reg[1]], [reg[4], reg[3]])
    squin.broadcast.sqrt_y_adj(reg)

    measurements = gemini_logical.terminal_measure(reg)

    for i in range(len(reg)):
        set_detector(measurements[i])
        set_observable(measurements[i], i)

In [3]:
service = GeminiLogical()
future = service.submit(main, shots=100)
results = future.get_results()

In [4]:
results.logical_bits

array([[False,  True, False,  True,  True],
       [ True, False,  True,  True,  True],
       [False, False, False,  True,  True],
       [ True,  True,  True, False, False],
       [ True,  True,  True,  True, False],
       [ True,  True,  True,  True, False],
       [ True, False,  True, False,  True],
       [False,  True, False,  True,  True],
       [False, False, False,  True, False],
       [False, False,  True, False, False],
       [ True,  True,  True, False,  True],
       [ True, False,  True,  True,  True],
       [False, False, False,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True, False, False],
       [False, False, False,  True, False],
       [False, False,  True,  True, False],
       [ True,  True, False,  True,  True],
       [ True,  True,  True, False, False],
       [False,  True, False, False,  True],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False,  True,  True,  Tr

In [5]:
decoder = TesseractDecoder(results.detector_error_model)

In [6]:
corrected_logical_bits = decoder.decode(results.detector_bits) ^ results.logical_bits
corrected_logical_bits

array([[False,  True, False,  True,  True],
       [ True, False,  True,  True,  True],
       [False,  True,  True,  True, False],
       [False,  True, False, False,  True],
       [ True, False, False, False,  True],
       [ True, False,  True, False,  True],
       [False, False, False, False,  True],
       [False,  True,  True, False, False],
       [False, False, False,  True, False],
       [False,  True, False, False, False],
       [False,  True, False, False,  True],
       [False, False, False,  True,  True],
       [False,  True,  True,  True, False],
       [ True,  True, False,  True,  True],
       [False, False,  True,  True, False],
       [ True, False,  True,  True,  True],
       [ True, False,  True,  True, False],
       [False, False, False, False,  True],
       [ True, False, False,  True, False],
       [ True,  True, False,  True, False],
       [False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False,  True,  Tr

## Underlying pipeline

In [7]:
program = compile_to_physical_stim_program(main)

In [8]:
print(program)

U3(0.0, 0.0, 0.03978874) 6
SQRT_Y_DAG 0 1 2 3 4 5
CZ 1 0 3 2 5 4
SQRT_Y 6
CZ 0 3 2 5 4 6
SQRT_Y 2 3 4 5 6
CZ 0 1 2 3 4 5
SQRT_Y 1 2 4
U3(0.0, 0.0, 0.03978874) 13
SQRT_Y_DAG 7 8 9 10 11 12
CZ 8 7 10 9 12 11
SQRT_Y 13
CZ 7 10 9 12 11 13
SQRT_Y 9 10 11 12 13
CZ 7 8 9 10 11 12
SQRT_Y 8 9 11
U3(0.0, 0.0, 0.03978874) 20
SQRT_Y_DAG 14 15 16 17 18 19
CZ 15 14 17 16 19 18
SQRT_Y 20
CZ 14 17 16 19 18 20
SQRT_Y 16 17 18 19 20
CZ 14 15 16 17 18 19
SQRT_Y 15 16 18
U3(0.0, 0.0, 0.03978874) 27
SQRT_Y_DAG 21 22 23 24 25 26
CZ 22 21 24 23 26 25
SQRT_Y 27
CZ 21 24 23 26 25 27
SQRT_Y 23 24 25 26 27
CZ 21 22 23 24 25 26
SQRT_Y 22 23 25
U3(0.0, 0.0, 0.03978874) 34
SQRT_Y_DAG 28 29 30 31 32 33
CZ 29 28 31 30 33 32
SQRT_Y 34
CZ 28 31 30 33 32 34
SQRT_Y 30 31 32 33 34
CZ 28 29 30 31 32 33
SQRT_Y 29 30 32
SQRT_X_DAG 0 1 2 3 4 5 6 7 8 9 10 11 12 13 28 29 30 31 32 33 34
PAULI_CHANNEL_1(0.0004102, 0.0004102, 0.0004112) 0 1 2 3 4 5 6 7 8 9 10 11 12 13 28 29 30 31 32 33 34
I_ERROR[loss](0) 0 1 2 3 4 5 6 7 8 9 10 11

In [9]:
program.diagram("timeline-svg", height=400)

In [10]:
dem = program.detector_error_model(approximate_disjoint_errors=True)
dem

stim.DetectorErrorModel('''
    error(0.00841086) D0 D1 D2
    error(0.00109209) D0 D1 D2 D3 D4 D5
    error(0.00230601) D0 D1 D2 D3 D4 D5 D6 D7 D8
    error(0.000285679) D0 D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 D11
    error(0.000285679) D0 D1 D2 D3 D4 D5 D9 D10 D11
    error(0.000285679) D0 D1 D2 D3 D4 D5 D9 D10 D11 D12 D13 D14
    error(0.00150156) D0 D1 D2 D3 D4 D5 D12 D13 D14
    error(0.00233947) D0 D1 D2 D6 D7 D8
    error(0.00014286) D0 D1 D2 D6 D7 D8 D9 D10 D11
    error(0.00014286) D0 D1 D2 D6 D7 D8 D9 D10 D11 D12 D13 D14
    error(0.00370878) D0 D1 D2 D6 D7 D8 D12 D13 D14
    error(0.00278893) D0 D1 D2 D9 D10 D11
    error(0.00307301) D0 D1 D2 D9 D10 D11 D12 D13 D14
    error(0.0122978) D0 D1 D2 D12 D13 D14
    error(0.000285679) D0 D1 D3 D4 D6 D7 D9 D10 L0 L1 L2 L3
    error(0.000285679) D0 D1 D3 D4 D9 D10 D12 D13 L0 L1 L3 L4
    error(0.000285679) D0 D1 D3 D4 D9 D10 L0 L1 L3
    error(0.00109209) D0 D1 D3 D4 L0 L1
    error(0.00014286) D0 D1 D6 D7 D9 D10 D12 D13 L0 L2 L3 L4
    e

In [11]:
sampler = program.compile_detector_sampler()

In [12]:
dets, obs =sampler.sample(shots=100_000, separate_observables=True)
dets

array([[False, False, False, ...,  True, False, False],
       [ True, False,  True, ..., False,  True,  True],
       [ True,  True,  True, ..., False, False,  True],
       ...,
       [False, False,  True, ...,  True, False,  True],
       [ True,  True, False, ...,  True, False, False],
       [False, False, False, ...,  True,  True, False]],
      shape=(100000, 15))

## Memory Experiment